In [1]:
from pycaret.datasets import get_data
from functools import partial
import pandas as pd
import time
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestRegressor as rfr
import shap
import xgboost as xgb
import numpy as np
from sklearn.feature_selection import RFE, SelectKBest, f_regression, mutual_info_regression
from sklearn.linear_model import RidgeClassifier, Ridge
from sklearn.model_selection import train_test_split


### Models

In [2]:
data = get_data('diabetes')
data_reg = get_data('insurance')
new_data = data.drop("Class variable", axis=1)
charges = data_reg.loc[data.index]["charges"]

,Number of times pregnant,Plasma glucose concentration a 2 hours in an oral glucose tolerance test,Diastolic blood pressure (mm Hg),Triceps skin fold thickness (mm),2-Hour serum insulin (mu U/ml),Body mass index (weight in kg/(height in m)^2),Diabetes pedigree function,Age (years),Class variable
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [3]:
labels = pd.read_csv("../data/regression_labels.csv", index_col=0)
regre = pd.read_csv("../data/regression.csv", index_col=0)

## Regression

In [2]:
import BioML.models.regression as regression
from collections import defaultdict
from functools import partial
import pandas as pd
import numpy as np
from pycaret.regression import RegressionExperiment
from scipy.stats import pearsonr

In [5]:
plot = ("learning", "error", "residuals")

In [6]:
excel = pd.read_excel("../data/esterase_features.xlsx", index_col=0, sheet_name="ch2_20")
label = list(range(len(excel)))

In [7]:
def pear_r(true_y, y_pred, **kwargs):
    correlate = pearsonr(true_y, y_pred)
    return correlate[0]
    

In [8]:
data = regression.DataParser(data_reg)
experiment = regression.PycaretInterface("regression", 200, budget_time=20, best_model=3, 
                                        output_path="regression_results", optimize="NDCG", 
                                        log_experiment=False)
regressor = regression.Regressor(optimize="NDCG", selected=["ridge", "xgboost", "rf"])
training = regression.Trainer(experiment, regressor, 5)


09-06-2024 11:49:06 INFO ------------------------------------------------------------------------------
09-06-2024 11:49:06 INFO PycaretInterface parameters
09-06-2024 11:49:06 INFO Seed: 200
09-06-2024 11:49:06 INFO Budget time: 20
09-06-2024 11:49:06 INFO The number of models to select: 3
09-06-2024 11:49:06 INFO Output path: regression_results
09-06-2024 11:49:06 INFO ----------------Trainer inputs-------------------------
09-06-2024 11:49:06 INFO Number of kfolds: 5
09-06-2024 11:49:06 INFO Number of retuning iterations: 50
09-06-2024 11:49:06 INFO Test size: 0.2


In [9]:
c = regression.split.ClusterSpliter("../data/resultsDB_clu.tsv", 5, random_state=experiment.seed)
X_train, X_test = c.train_test_split(data.features)

ValueError: Found input variables with inconsistent numbers of samples: [1338, 0]

In [9]:
results, models_dict = training.generate_training_results(data_reg, "charges", tune=False)

09-06-2024 11:49:11 INFO --------------------------------------------------------
09-06-2024 11:49:11 INFO Training regression models
09-06-2024 11:49:11 INFO The models used ['ridge', 'xgboost', 'rf']
09-06-2024 11:49:11 INFO The number of models used 3
09-06-2024 11:49:11 INFO Time budget is 20 minutes
09-06-2024 11:49:14 INFO Model ridge trained in 0.051 minutes
09-06-2024 11:49:17 INFO Model xgboost trained in 0.043 minutes
09-06-2024 11:49:19 INFO Model rf trained in 0.042 minutes
09-06-2024 11:49:19 INFO Training over: Total runtime 0.136 minutes
09-06-2024 11:49:19 INFO Analyse the best models and plotting them
09-06-2024 11:49:19 INFO Analyse the top 1 model: ridge
09-06-2024 11:49:21 INFO Analyse the top 2 model: rf
09-06-2024 11:49:25 INFO Analyse the top 3 model: xgboost
09-06-2024 11:49:28 INFO --------Stacking the best models--------
09-06-2024 11:49:28 INFO ----------Stacking the best models--------------
09-06-2024 11:49:31 INFO --------Creating an ensemble model--------

In [49]:
x_train = experiment.pycaret.get_config("X_train")
y_train = experiment.pycaret.get_config("y_train")

225     11987.167969
420     46889.261719
853     11729.679688
289     25992.820312
147      9877.607422
            ...     
1103    11363.283203
844     10072.054688
810      9414.919922
784      4350.514160
617     23306.546875
Name: charges, Length: 1070, dtype: float32

In [56]:
X_train = experiment.pycaret.get_config("X_train_transformed")

In [58]:
correlate = pearsonr(y_train, pred)

In [71]:
correlate

PearsonRResult(statistic=0.8638410477076413, pvalue=2.6917e-320)

In [10]:
results["not_tuned"]["train"][0]

MAE           MSE       RMSE      R2   RMSLE  \
        Split    Fold                                                       
ridge   CV-Train Mean  4126.8902  3.638525e+07  6031.0584  0.7468  0.5797   
                 Std    113.0031  1.295802e+06   107.6196  0.0054  0.0228   
        CV-Val   Mean  4179.4594  3.728019e+07  6091.8197  0.7393  0.5920   
                 Std    365.6410  5.046651e+06   412.2174  0.0216  0.1040   
rf      CV-Train Mean  1022.6477  3.422717e+06  1849.3452  0.9762  0.1797   
                 Std     48.6998  1.868239e+05    51.3737  0.0014  0.0071   
        CV-Val   Mean  2757.5890  2.430746e+07  4909.7675  0.8303  0.4620   
                 Std    356.0923  4.369858e+06   449.0432  0.0220  0.0475   
xgboost CV-Train Mean   337.0519  4.218450e+05   642.9434  0.9971  0.0912   
                 Std     34.7308  1.186714e+05    92.0265  0.0009  0.0145   
        CV-Val   Mean  3233.9792  3.151363e+07  5590.9190  0.7788  0.5717   
                 Std    324.8764  5.633689e+06   505.2262  0.0359  0.0775   

                         MAPE    NDCG  Pearson  
        Split    Fold                           
ridge   CV-Train Mean  0.4081  0.8458   0.8642  
                 Std   0.0171  0.0522   0.0031  
        CV-Val   Mean  0.4136  0.8972   0.8621  
                 Std   0.0516  0.0568   0.0122  
rf      CV-Train Mean  0.1204  0.9934   0.9885  
                 Std   0.0083  0.0031   0.0007  
        CV-Val   Mean  0.3275  0.8941   0.9124  
                 Std   0.0467  0.0666   0.0119  
xgboost CV-Train Mean  0.0504  0.9991   0.9986  
                 Std   0.0056  0.0014   0.0004  
        CV-Val   Mean  0.4155  0.8665   0.8869  
                 Std   0.0662  0.1158   0.0186

In [7]:
test_set_predictions = training.generate_holdout_prediction(models_dict)

In [9]:
training_output = "regression_results"
for tune_status, result_dict in results.items():
    for key, value in result_dict.items():
        regression.write_results(f"{training_output}/{tune_status}", *value, sheet_name=key)
    regression.write_results(f"{training_output}/{tune_status}", test_set_predictions[tune_status] , sheet_name=f"test_results")

## Save models from pre-trained

In [ ]:
from BioML.models import save_model

In [ ]:
generate = save_model.GenerateModel(training)
final_model = generate.finalize_model(models["tuned"]["majority"])
generate.save_model(final_model, "model_output/majority")

## Save Models from scratch

In [11]:
from BioML.models import save_model

In [6]:
data = save_model.DataParser("../data/regression.csv", "../data/regression_labels.csv")
experiment = save_model.PycaretInterface("regression", 200, budget_time=20, best_model=3, 
                                  output_path="regression_training", optimize="RMSE", experiment_name="generate_model")
regressor = save_model.Regressor(test_size=0.2, optimize="RMSE", selected=["br", "lr"])
training = save_model.Trainer(experiment, regressor, 5, 30)


23-02-2024 12:21:03 INFO ------------------------------------------------------------------------------
23-02-2024 12:21:03 INFO PycaretInterface parameters
23-02-2024 12:21:03 INFO Seed: 200
23-02-2024 12:21:03 INFO Budget time: 20
23-02-2024 12:21:03 INFO The number of models to select: 3
23-02-2024 12:21:03 INFO Output path: regression_training
23-02-2024 12:21:03 INFO ----------------Trainer inputs-------------------------
23-02-2024 12:21:03 INFO Number of kfolds: 5
23-02-2024 12:21:03 INFO Number of iterations: 30


In [7]:
regressor.selected

['br', 'lr']

In [10]:
sorted_results, sorted_models, top_params = training.run_training(data.features, data.label)

2024/02/23 12:22:41 INFO mlflow.tracking.fluent: Experiment with name 'generate_model' does not exist. Creating a new experiment.
23-02-2024 12:22:49 INFO --------------------------------------------------------
23-02-2024 12:22:49 INFO Training regression models
23-02-2024 12:22:49 INFO The models used ['br', 'lr']
23-02-2024 12:22:49 INFO Time budget is 20 minutes
23-02-2024 12:23:34 INFO Training over: Total runtime 0.765 minutes
23-02-2024 12:23:34 INFO Analyse the best models and plotting them
23-02-2024 12:23:34 INFO Analyse the top 1 model: br
23-02-2024 12:23:41 INFO Analyse the top 2 model: lr


In [28]:
generate = save_model.GenerateModel(training)
models =  generate.train_by_strategy(sorted_models, "majority")
final_model = generate.finalize_model(models)
generate.save_model(final_model, "model_output/majority")

26-12-2023 13:40:52 INFO --------Creating an ensemble model--------
26-12-2023 13:40:52 INFO ----------Creating a majority voting model--------------
26-12-2023 13:40:52 INFO fold: 5
26-12-2023 13:40:52 INFO weights: None
26-12-2023 13:40:53 INFO ----------Finalizing the model by training it with all the data including test set--------------


Transformation Pipeline and Model Successfully Saved


## Prediction

In [1]:
from BioML.models import predict
import pandas as pd
import numpy as np
from scipy.spatial import distance

In [11]:
training_features = "../data/regression.csv"
label = pd.read_csv("../data/regression_labels.csv", index_col=0)
test_features = "../data/regression.csv"
outlier_train=()
outlier_test=()
sheet_name=None
problem="regression"
model_path="model_output/majority"
scaler="zscore"

In [12]:
feature = predict.DataParser(training_features, label, outliers=outlier_train, sheets=sheet_name)
test_features = feature.remove_outliers(feature.read_features(test_features), outlier_test)
predictions = predict.predict(test_features, model_path, problem)

In [5]:
transformed, scaler_dict, test_x = predict.scale(scaler, feature.drop(), test_features)

In [27]:
filtered_pred = predict.domain_filter(predictions, transformed, test_x, 1)

In [ ]:
col_name = ["prediction_score", "prediction_label", "AD_number"]
predictions = predictions.loc[:, predictions.columns.str.contains("|".join(col_name))] 

## Test

In [14]:
import operator as op
import pandas as pd
import numpy as np

In [ ]:
def calculate_score_dataframe(dataframe: pd.DataFrame, 
                              optimize="Pearson", 
                              greater_is_better=True) -> float | int: # type: ignore
    """
    Calculates the score of a DataFrame based on the optimization metric and R2 score.

    Parameters
    ----------
    dataframe : pd.DataFrame
        The DataFrame containing the performance metrics of a regression model.

    Returns
    -------
    int or float
        The calculated score of the DataFrame.


    Examples
    --------
    >>> dataframe = pd.read_csv("performance_metrics.csv")
    >>> score = _calculate_score_dataframe(dataframe)
    ... # Calculates the score of the DataFrame based on the optimization metric and R2 score.
    """
    cv_train = dataframe.loc[("CV-Train", "Mean")]
    cv_val = dataframe.loc[("CV-Val", "Mean")]

    operation = op.add
    penalize = 1e6 if cv_train[optimize] == 1 else 0
    if greater_is_better:
        operation = op.sub
        penalize = -1e6 if cv_train[optimize] == 1 else 0

    metric = cv_train[optimize] * 0.7 + cv_val[optimize] # type: ignore
    diff =  1.2 * abs(cv_val[optimize] - cv_train[optimize]) # type: ignore
    rmse = operation(metric, diff) + penalize
    if greater_is_better:
        return rmse
    return - rmse # negative because the sorting is from greater to smaller, in this case the smaller the error value the better

In [32]:
sheet = pd.ExcelFile("/home/ruite/Projects/horus_bioml/ph_features/regression_results/training_results.xlsx").sheet_names
data = pd.read_excel("/home/ruite/Projects/horus_bioml/ph_features/regression_results/training_results.xlsx", sheet_name=sheet, index_col=[0,1,2])
performance_list = []
for sheet, sorted_results in data.items():
    index = sorted_results.index.unique(0)
    score = np.nanmean([calculate_score_dataframe(sorted_results.loc[i], "RMSE", False) for i in index])

    performance_list.append((sheet, sorted_results.loc[index], score/len(index)))
    performance_list.sort(key=lambda x: x[2], reverse=True) # the greater the better

In [33]:
from BioML.utilities.utils import write_results
for sheet, performance, score in performance_list:
    write_results("/home/ruite/Projects/horus_bioml/ph_features/regression_3", performance, sheet_name=sheet)